# Test Face Diarization with OpenFace

In [1]:
from moviepy.editor import VideoFileClip
import subprocess
import os
import IPython
import docker
import pandas as pd

client = docker.from_env()


In [3]:
SAVE_DIR = "video_debate_21"
VIDEO_FILE = "debate_2021_compressed.mp4"


In [4]:
def extract_features_video_docker(client, filename, logfilename):
    """ 
    Extracts openface features from a single face in a video file using a docker container. 
    Copies video file into the openface docker container.
    Extracts openface features in docker container.
    Saves openface log to a logfile.
    Copies results (processed files) back to working directory. 
    """
    container = client.containers.run(
        "algebr/openface:latest", name="openface", stdin_open=True, tty=True, remove=True, detach=True)
    print("Started docker container")

    with open(logfilename, "wb") as logfile:
        subprocess.run(
            f"docker cp {filename} openface:/home/openface-build", shell=True)
        print("Copied file to docker container")
        _, output = container.exec_run(
            f"build/bin/FeatureExtraction -f {filename.split('/')[-1]}")
        print("Extracted openface features")
        subprocess.run(
            "docker cp openface:/home/openface-build/processed .", shell=True)
        print("Copied results from docker container to /processed")
        logfile.write(output)

    container.stop()
    print("Stopped docker container")


In [7]:
if not os.path.isfile(f"{SAVE_DIR}/debate_21_short.mp4"):
    with VideoFileClip(f"{SAVE_DIR}/{VIDEO_FILE}") as clip:
        short_clip = clip.subclip(30*60, 35*60)
        short_clip.write_videofile(f"{SAVE_DIR}/debate_21_short.mp4")


Moviepy - Building video debate_21_short.mp4.
MoviePy - Writing audio in debate_21_shortTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video debate_21_short.mp4



Moviepy - Done !
Moviepy - video ready debate_21_short.mp4


In [8]:
extract_features_video_docker(
    client, f"{SAVE_DIR}/debate_21_short.mp4", f"{SAVE_DIR}/debate_21_short.log")


Started docker container
Copied file to docker container
Extracted openface features
Copied results from docker container to /processed
Stopped docker container


In [2]:
features = pd.read_csv("processed/debate_21_short.csv")


In [3]:
features


,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.00,0.00,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0.04,0.00,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0.08,0.00,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0.12,0.00,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0.16,0.00,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,7496,0,299.80,0.98,1,0.102659,0.262200,-0.959537,0.041624,0.312898,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
7496,7497,0,299.84,0.98,1,0.185593,0.233249,-0.954542,0.170350,0.274257,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
7497,7498,0,299.88,0.98,1,0.180517,0.231645,-0.955905,0.176982,0.285691,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
7498,7499,0,299.92,0.98,1,0.093828,0.195459,-0.976213,0.020585,0.291756,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
set(features.iloc[:, 1])


{0}